<h1>Trabajdo Realizado para la Feria</h1>

<h2>Integrantes</h2>
<p>Luis Caballero 8-9231440<br>
Ángel Cilli<br>
Jaime Acosta</p>

<p><b>Idea o Concepto:</b>Trabajaremos un reconocedor de "personal" dentro del equipo, un Aruco distinto para Jaime Acosta y Luis Caballero, que cuando sean mostrados a la cámara el programa ejecutará distintos enfoques con Aruco.</p>